In [19]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from PIL import Image
from train_pm import Dataset, double_conv, LeUNet, val
import cv2

In [20]:
model = LeUNet()
model = torch.nn.DataParallel(model).cuda()
model.load_state_dict(torch.load("model_hazy_best.pth"),strict=False) # on GPU
criterion = nn.MSELoss().cuda()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)

In [21]:
transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize(mean=[0.5231, 0.5180, 0.5115],std=[0.2014, 0.2018, 0.2100]),])

In [22]:
cv_in = cv2.imread('./test_delhi_1.jpg',1) # color
pil_in = Image.fromarray(cv_in)
X = transform(pil_in)

In [23]:
X = X[None, :, :, :]

In [27]:
x = X.cuda(non_blocking=True)
x_var = torch.autograd.Variable(x)

In [29]:
model(x_var)

/opt/anaconda/envs/py3cv/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/opt/anaconda/envs/py3cv/lib/python3.8/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


tensor([[-0.0732]], device='cuda:0', grad_fn=<AddmmBackward>)

In [28]:
x_var.shape

torch.Size([1, 3, 224, 224])